# Análisis de Rendimiento Estudiantil 
### 1. Exploración inicial de la información 

In [4]:
import pandas as pd
import numpy as np

In [5]:
#CA_01: Cargar la base de datos de estudiantes. 
df = pd.read_csv("student-mat.csv", sep=";")
display(df.head())

# CA_02: Seleccionar columnas 
df_columns = df[["age", "studytime", "failures", "absences", "G1", "G2", "G3"]].copy()
display(df_columns.head())

#CA_03: Filtrar y mostrar estudiantes mayores de 18 años
df_mayores = df_columns["age"] > 18
print("Estudiantes mayores de 18 años:")
display(df_columns[df_mayores])

#CA_04: Filtrar y mostrar estudiantes con más de 20 ausencias
df_ausentes = df_columns["absences"] > 20
print("Estudiantes con más de 20 ausencias:")
display(df_columns[df_ausentes])

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


,age,studytime,failures,absences,G1,G2,G3
0,18,2,0,6,5,6,6
1,17,2,0,4,5,5,6
2,15,2,3,10,7,8,10
3,15,3,0,2,15,14,15
4,16,2,0,4,6,10,10


Estudiantes mayores de 18 años:


,age,studytime,failures,absences,G1,G2,G3
127,19,2,3,2,7,8,9
153,19,1,3,0,5,0,0
210,19,4,0,10,8,8,8
247,22,1,3,16,6,8,8
257,19,2,0,12,11,11,11
270,19,2,2,15,9,9,9
296,19,2,0,0,10,9,0
304,19,2,1,20,15,14,13
306,20,1,0,0,17,18,18
307,19,1,1,38,8,9,8


Estudiantes con más de 20 ausencias:


,age,studytime,failures,absences,G1,G2,G3
40,16,2,1,25,7,10,11
74,16,2,0,54,11,12,11
103,15,2,0,26,7,6,6
183,17,2,0,56,9,9,8
198,17,1,1,24,18,18,18
205,17,3,1,28,10,9,9
216,17,2,2,22,6,6,4
260,18,2,0,21,17,18,18
276,18,2,0,75,10,9,9
277,18,1,0,22,9,9,9


### 2. Revisión de calidad de los datos

In [6]:
# CA_05: Identificar y eliminar registros repetidos. 
duplicados = df_columns.duplicated().sum()
print("Número de registros duplicados:", duplicados)
df_sin_duplicados = df_columns.drop_duplicates()

# CA_06:  Detectar valores extremos en (absences y G3) notas finales - (outliers) 
# Calcular outliers en ausencias
Q1 = df_columns["absences"].quantile(0.25)  # Primer cuartil (25%)
Q3 = df_columns["absences"].quantile(0.75)  # Tercer cuartil (75%)
IQR = Q3 - Q1                              # Rango intercuartílico
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

outliers_absences = df_columns[
    (df_columns["absences"] < limite_inferior) | (df_columns["absences"] > limite_superior)
]
print("Outliers en ausencias:")
display(outliers_absences)

# Calcular outliers en nota final G3
Q1_g3 = df_columns["G3"].quantile(0.25)
Q3_g3 = df_columns["G3"].quantile(0.75)
IQR_g3 = Q3_g3 - Q1_g3
limite_inf_g3 = Q1_g3 - 1.5 * IQR_g3
limite_sup_g3 = Q3_g3 + 1.5 * IQR_g3

outliers_g3 = df_columns[
    (df_columns["G3"] < limite_inf_g3) | (df_columns["G3"] > limite_sup_g3)
]
print("Outliers en notas finales (G3):")
display(outliers_g3)


# CA_07: Validar que las notas (G1, G2, G3) estén dentro del rango oficial 0–20. 
notas_fuera_rango = df_columns[
    (df_columns["G1"] < 0) | (df_columns["G1"] > 20) |
    (df_columns["G2"] < 0) | (df_columns["G2"] > 20) |
    (df_columns["G3"] < 0) | (df_columns["G3"] > 20)
]

print("Notas fuera del rango 0-20:")
display(notas_fuera_rango)

Número de registros duplicados: 6
Outliers en ausencias:


,age,studytime,failures,absences,G1,G2,G3
40,16,2,1,25,7,10,11
74,16,2,0,54,11,12,11
103,15,2,0,26,7,6,6
183,17,2,0,56,9,9,8
198,17,1,1,24,18,18,18
205,17,3,1,28,10,9,9
216,17,2,2,22,6,6,4
260,18,2,0,21,17,18,18
276,18,2,0,75,10,9,9
277,18,1,0,22,9,9,9


Outliers en notas finales (G3):


,age,studytime,failures,absences,G1,G2,G3


Notas fuera del rango 0-20:


,age,studytime,failures,absences,G1,G2,G3


### 3. Indicadores básicos de rendimiento

In [7]:
# CA_07: Promedio, mínimo y máximo de la nota final (G3).
promedio = df_columns["G3"].mean()
minimo = df_columns["G3"].min()
maximo = df_columns["G3"].max()
print("Promedio de nota final (G3):", promedio)
print("Nota mínima (G3):", minimo)
print("Nota máxima (G3):", maximo)

# CA_08: Cantidad de estudiantes por nivel de horas de estudio.
estudiantes = df_columns["studytime"].value_counts().sort_index()

print("Cantidad de estudiantes por nivel de horas de estudio:")
display(estudiantes)

# CA_09: Promedio general de ausencias.
promedio_ausencias = df_columns["absences"].mean()
print("Promedio general de ausencias:", promedio_ausencias)

Promedio de nota final (G3): 10.415189873417722
Nota mínima (G3): 0
Nota máxima (G3): 20
Cantidad de estudiantes por nivel de horas de estudio:


studytime
1    105
2    198
3     65
4     27
Name: count, dtype: int64

Promedio general de ausencias: 5.708860759493671


### 4.  Indicadores complementarios

In [8]:
# CA_11: Calcular aprobados y reprobados
aprobados = (df_columns["G3"] >= 10).sum()
reprobados = (df_columns["G3"] < 10).sum()
total = len(df_columns) 
porcentaje_aprobados = (aprobados / total) * 100
porcentaje_reprobados = (reprobados / total) * 100
print(f" Aprobados: {aprobados} ({porcentaje_aprobados:.2f}%)")
print(f" Reprobados: {reprobados} ({porcentaje_reprobados:.2f}%)") 

# CA_11.2: Clasificación de ausencias en niveles
def clasificar_ausencias(x):
    if x <= 5:  
        return "Bajo"
    elif x <= 15:
        return "Medio"
    else:
        return "Alto"

df_columns["nivel_ausencias"] = df_columns["absences"].apply(clasificar_ausencias)
# Contar estudiantes por nivel
distribucion_ausencias = df_columns["nivel_ausencias"].value_counts()
print("Distribución de ausencias:")
display(distribucion_ausencias)


# CA_11.3 Normalización Min-Max de la nota final G3
g3_min = df_columns["G3"].min()
g3_max = df_columns["G3"].max()

df_columns["G3_normalizado"] = (df_columns["G3"] - g3_min) / (g3_max - g3_min)

print("Notas finales normalizadas (primeras filas):")
display(df_columns[["G3", "G3_normalizado"]].head())

 Aprobados: 265 (67.09%)
 Reprobados: 130 (32.91%)
Distribución de ausencias:


nivel_ausencias
Bajo     249
Medio    113
Alto      33
Name: count, dtype: int64

Notas finales normalizadas (primeras filas):


,G3,G3_normalizado
0,6,0.30
1,6,0.30
2,10,0.50
3,15,0.75
4,10,0.50
